In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('C:/Users/yratz/OneDrive/מסמכים/לימודים/Data analize/task2/matala2_cosmetics_2019-Nov.csv')
data.columns

In [ ]:
#1
data['user_session'] = data['user_session'].fillna(method='ffill')
data=data.sort_values(by='event_time')
data['event_time'] = pd.to_datetime(data['event_time'])
data['duration_to_next_event'] = data.groupby('user_session')['event_time'].apply(
    lambda x: (x.shift(-1)-x)).fillna(pd.Timedelta(seconds=0))
data['duration_to_next_event'] = data['duration_to_next_event'].apply(lambda x: x.total_seconds())
data

In [ ]:
#2
data['funnel_number'] = data.groupby('user_id')['event_time'].apply(
    lambda x: (x.dt.day - x.shift(1).dt.day> 5).cumsum()+1)
data

In [ ]:
#3
data['index_in_funnel'] = data.groupby(['user_id','funnel_number'])['user_session'].apply(lambda x: (x != x.shift()).cumsum())


In [ ]:
#4
import re
data['price'] = data['price'].astype(str)
data['price']=data['price'].apply(lambda x: float(re.findall('[0-9]+\.+[0-9]+|[0-9]+',x)[0]))
data['price'].dtype

In [ ]:
#5
import matplotlib.pyplot as plt
group_data=data.groupby('event_type').count()
plt.bar(group_data.index, group_data['user_id'])
plt.xlabel('Event type')
plt.ylabel('Number of operations')
plt.show()

In [ ]:
#6
grouped_data = data.groupby('user_session')
session_data = pd.DataFrame()
session_data['User_session'] = grouped_data['user_session'].unique().apply(lambda x: x[0])
session_data['User_id'] = grouped_data['user_id'].agg('first')
session_data['funnel_number'] = grouped_data['funnel_number'].agg('first')
session_data['index_in_funnel'] = grouped_data['index_in_funnel'].agg('last')
session_data['num_of_events'] = grouped_data['event_type'].count()
session_data['duration_in_sec'] = grouped_data['duration_to_next_event'].sum()
session_data['list_of_viewed'] = grouped_data.apply(lambda x: x.loc[x['event_type']=='view','product_id'].tolist())
session_data['list_of_added_to_cart'] = grouped_data.apply(lambda x: x.loc[x['event_type']=='cart','product_id'].tolist())
session_data['list_of_purchased'] = grouped_data.apply(lambda x: x.loc[x['event_type']=='purchased','product_id'].tolist())
session_data=session_data.reset_index(drop=True)
session_data